In [ ]:
# just getting seaerch and search id
import requests
import json

def search_divar():

    url = "https://api.divar.ir/v8/postlist/w/search"

    query = "دزدگیر"
    sort_values = ["cheapest", "sort_date", "most_expensive"]

    payload = {
        "city_ids": ["1"],
        "source_view": "SEARCH",
        "disable_recommendation": False,
        "map_state": {"camera_info": {"bbox": {}}},
        "search_data": {
            "form_data": {
                "data": {
                    "category": {"str": {"value": "parts-accessories"}}
                }
            },
            "server_payload": {
                "@type": "type.googleapis.com/widgets.SearchData.ServerPayload",
                "additional_form_data": {
                    "data": {
                        "sort": {"str": {"value": "sort_date"}}
                    }
                }
            },
            "query": query
        }
    }

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://divar.ir/",
        "X-Screen-Size": "744x963",
        "X-Standard-Divar-Error": "true",
        "X-Render-Type": "CSR",
        "Origin": "https://divar.ir"
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        data = response.json()
        print(json.dumps(data, indent=2, ensure_ascii=False))
        print(data["action_log"]["server_side_info"]["info"]["search_id"])
        return data["action_log"]["server_side_info"]["info"]["search_id"]
    else:
        print(f"Error: status {response.status_code}")
        print(response.text)
        return None

search_divar()



In [ ]:
# right one 

import requests
import json
import os
import time

proxies = {
    "http": "http://127.0.0.1:12334",
    "https": "http://127.0.0.1:12334",
}

url = "https://api.divar.ir/v8/postlist/w/search"
headers = {
    "Content-Type": "application/json"
}

bad_response_count = 0
city_id = 1

while bad_response_count < 10:
    print(f"\nChecking city_id: {city_id}")
    successful_city = False

    for page in range(0, 151):
        payload = {
            "city_ids": [str(city_id)],
            "pagination_data": {
                "@type": "type.googleapis.com/post_list.PaginationData",
                "last_post_date": "2025-05-08T10:39:16.934211626Z",
                "page": page,
                "layer_page": 1,
                "search_uid": "16676474-66dd-4a57-826a-e802656b05b1"
            },
            "disable_recommendation": False,
            "map_state": {"camera_info": {"bbox": {}}},
            "search_data": {
                "form_data": {
                    "data": {
                        "sort": {"str": {"value": "most_expensive"}},
                        "category": {"str": {"value": "temporary-rent"}}
                    }
                },
                "server_payload": {
                    "@type": "type.googleapis.com/widgets.SearchData.ServerPayload",
                    "additional_form_data": {}
                }
            }
        }

        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload), proxies=proxies)
        except Exception as e:
            print(f"Error with city_id {city_id}, page {page}: {e}")
            break

        if response.status_code != 200:
            print(f"Bad response (status {response.status_code}) for city_id {city_id}, page {page}")
            if page == 0:
                bad_response_count += 1
            break
        else:
            successful_city = True

        data = response.json()
        widgets = data.get("list_widgets", [])
        print(f"Page {page} returned {len(widgets)} widgets")

        for i, widget in enumerate(widgets):
            try:
                web_info = widget["data"]["action"]["payload"]["web_info"]
                city_name = web_info.get("city_persian", f"city_{city_id}")
                district_name = web_info.get("district_persian", None)

                root_dir = os.path.join("test", city_name)
                if district_name:
                    root_dir = os.path.join(root_dir, district_name)

                os.makedirs(root_dir, exist_ok=True)

                file_path = os.path.join(root_dir, f"item_{page}_{i}.json")
                with open(file_path, "w", encoding="utf-8") as f:
                    json.dump(widget, f, ensure_ascii=False, indent=2)

            except Exception as e:
                root_dir = os.path.join("error", city_name)
                if district_name:
                    root_dir = os.path.join(root_dir, district_name)

                os.makedirs(root_dir, exist_ok=True)

                file_path = os.path.join(root_dir, f"item_{page}_{i}.json")
                with open(file_path, "w", encoding="utf-8") as f:
                    json.dump(widget, f, ensure_ascii=False, indent=2)

        time.sleep(3)  
    if successful_city:
        bad_response_count = 0  

    city_id += 1
